In [2]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="nebiyu29/fintunned-v2-roberta_GA", device='cpu')

In [3]:
print(f"classification classes: {list(pipe.model.config.id2label.values())}")

classification classes: ['depression', 'anxiety', 'bipolar disorder', 'schizophrenia', 'PTSD', 'OCD', 'ADHD', 'autism', 'eating disorder', 'personality disorder', 'phobia']


In [4]:
import time
start_time = time.time()
output = pipe("I can't control myself eating chocolate")
print(*output, sep='\n')
print(f"pred time {(time.time() - start_time) * 1000}ms")

{'label': 'eating disorder', 'score': 0.9883596897125244}
pred time 762.8328800201416ms


In [5]:
def choose_specialist(predicted_field, specialists_priorities, thresh=0.5):
    field = predicted_field['label']
    score = predicted_field['score']
    if score < thresh:
        return None
    related_specialists = []
    for spec in specialists_priorities:
        if field in spec["priorities"].keys():
            related_specialists.append((spec["name"], spec["priorities"][field]))
    sorted_related_specialists = sorted(related_specialists, key=lambda x: x[1])

    if sorted_related_specialists:
        return sorted_related_specialists[0]
    return None

In [6]:
specialists_priorities = [{
    "name": "Михаил Исаков",
    "priorities": {"PTSD": 1, "eating disorder": 2},
},
{
    "name": "Головаха Николай",
    "priorities": {"depression": 1, "anxiety": 2, "phobia": 3},
},
{
    "name": "Глебова Мария",
    "priorities": {"OCD": 1, "personality disorder": 2},
}
]
choose_specialist(output[0], specialists_priorities)

('Михаил Исаков', 2)